In [50]:
import pandas as pd
import numpy as np
import random
import re
from tqdm import tqdm
import enum
import copy

In [115]:
DEFAULT_NUM_YEARS = 50
DEFAULT_FIGHTS_PER_YEAR = 12
DEFAULT_NUM_FIGHTERS = 100
DEFAULT_PARTY_SIZE = 1
DEFAULT_PCT_MAGIC_PER_LEVEL = 15
DEFAULT_ARMOR = armType(1)

fighterlist = []
numYears = DEFAULT_NUM_YEARS
fightsPerYear = DEFAULT_FIGHTS_PER_YEAR
numFighters = DEFAULT_NUM_FIGHTERS
startLevel = 0
fighterPartySize = DEFAULT_PARTY_SIZE

fightManVsMonster = False
useMonsterTreasureType = False
useRevisedXPAwards = False

#** Report summary statistics? */
reportFighterStats = True

#** Report full details on each fighter? */
reportFighterData = False

#** Report kills achieved by each monster type? */
reportMonsterKills = False

#** Report total kills by monster level? */
reportTotalMonsterKills = False

#** Report status at each year-end? */
reportYearEnd = False

#** Report XP award ratios? */
reportXPAwards = False

#** Report each individual XP award? */
reportAllXPAwards = False

#** Report every encounter? */
reportEveryEncounter = False

#** Create win percent matrix? */
makeWinPercentMatrix = False

#** Age of oldest fighter who ever lived. */
supMaxAge = 120

#** Total awarded monster XP. */
totalMonsterXP = 0

#** Total awarded treasure XP. */
totalTreasureXP = 0



In [155]:
##Enumerations        
class armType(enum.Enum):
    Plate = 1
    Chain = 2
    Leather = 3
    Shield = 4

class Alignment(enum.Enum):
    Lawful = 1
    Neutral = 2
    Chaotic = 3
    
    #Convert a letter to alignment.
    @classmethod
    def getFromChar(cls, c):
        if c == "L" or c == "Lawful":
            return cls.Lawful
        if c == "N" or c == "Neutral":
            return cls.Neutral
        if c == "C" or c == "Chaotic":
            return cls.Chaotic

    #Randomize a normally-distributed alignment. 
    @classmethod
    def randomNormal(cls):
        d6 = random.randint(1,6)
        if d6 >=2 and d6 <=5:
            return cls.Neutral
        if d6 == 1:
            return cls.Lawful
        if d6 == 6:
            return cls.Chaotic

    #Randomize a uniformly-distributed alignment. 
    @classmethod
    def randomUniform(cls):
        d6 = random.randint(1,6)
        if d6 >=1 and d6 <=2:
            return cls.Lawful
        if d6 >=3 and d6 <=4:
            return cls.Neutral
        if d6 >=5 and d6 <=6:
            return cls.Chaotic

    #Randomize a Lawful-distributed alignment. 
    @classmethod
    def randomUniform(cls):
        d6 = random.randint(1,6)
        if d6 >=1 and d6 <=4:
            return cls.Lawful
        if d6 >=5:
            return cls.Neutral

    #Randomize a Chatoric-distributed alignment. 
    @classmethod
    def randomUniform(cls):
        d6 = random.randint(1,6)
        if d6 >=1 and d6 <=2:
            return cls.Neutral
        if d6 >=5:
            return cls.Chaotic
        
##Classes    
class Equipment:
    def __init__(self, name, magicBonus, weight):
        self.MAX_MAGIC_BONUS = 5
        self.ONE_THIRD = 1.0/3.0
        self.name = name
        self.magicBonus = magicBonus
        self.weight = weight
        
    #Set the magic bonus.
    def setMagicBonus(self, bonus):
        self.magicBonus = min(bonus, self.MAX_MAGIC_BONUS);
    
    #Increment the magic bonus.
    def incMagicBonus(self):
        setMagicBonus(self.magicBonus+1)

class Armor(Equipment):
    def __init__(self, armorType, baseArmor, weight, magicBonus):
        self.baseArmor = baseArmor
        self.armorType = armorType
        super().__init__(str(armType(armorType)), weight, magicBonus)
        #Enumeration
        
    def getArmorType(self):
        return self.armorType
    
    def getBaseArmor(self):
        return self.baseArmor
    
    def isMetal(self):
        if self.armorType == armType.Plate.value or self.armorType == armType.Chain.value:
            return True
        else:
            return False
            
    def copy(self):
        return copy.deepcopy(self)
    
    def makeType(armType):
        if armType == armType.Plate:
            return(Armor(armType, 6, 4, 0))
        if armType == armType.Chain:
            return(Armor(armType, 4, 2, 0))
        if armType == armType.Chain:
            return(Armor(armType, 2, 1, 0))
        if armType == armType.Chain:
            return(Armor(armType, 1, 1, 0))


class Monster:
    def __init__(self):
        
        self.UNDEFINED_EHD = -1
        self.BASE_HIT_DIE = 6
        self.MAX_MELEERS = 6
        
        #get from DataFrame in string constructor version pass df.loc row?
        self.race = ""
        self.sourceBook = ""
        self.monType = ""
        self.environment = ""
        self.numberAppearing = ""
        self.armorClass = 10
        self.moveInches = 0
        self.hitDice = 0
        self.inLairPct = 0
        self.treasureType = ""
        self.attack = None
        self.alignment = None
        self.hitDiceAsFloat = 0.0
        self.equivalentHitDice = 0
        self.hitPoints = 0
        self.maxHitPoints = 0
        self.dragonAge = 0
        self.breathCharges = 0
        self.killTally = 0
        self.timesMeleed = 0
        self.host = None
        self.specialList = set()
        self.conditionList = set()
        self.specialValues = {}  #set values???
#Set<SpecialType> specialList = None
#Set<SpecialType> conditionList = None
#AbstractMap<SpecialType, Integer> specialValues = None
        spellMemory = None
    
    def copy(self):
        return copy.deepcopy(self)
    
# Methods	--------------------------------------------------------------------------

	# Primary accessors
def getRace(self):
    return self.race
def getSourceBook(self):
    return self.sourceBook
def getNumberAppearing(self):
    return self.numberAppearing
def getArmorClass(self):
    return self.armorClass
def getHitPoints(self):
    return self.hitPoints
def getMaxHitPoints(self):
    return self.maxHitPoints
def getInLairPct(self):
    return self.inLairPct
def getHitDice(self):
    return self.hitDice
def getHitDiceNum(self):
    return self.hitDice.getNum()
def getLevel(self):
    return self.getHitDiceNum()
def getTreasureType(self):
    return self.treasureType
def getAttack(self):
    return self.attack
def getAlignment(self):
    return self.alignment
def getType(self):
    return self.monType
def getEquivalentHitDice(self):
    return self.equivalentHitDice
def getEnvironment(self):
    return self.environment
def getHitDiceAsFloat(self):
    return self.hitDiceAsFloat
def getKillTally(self):
    return self.killTally
def getTimesMeleed(self):
    return self.timesMeleed
def getHost(self):
    return self.host

#// Shortcut accessors
def getAC(self):
    return self.getArmorClass()
def getHD(self):
    return self.getHitDiceNum()
def getHP(self):
    return self.getHitPoints()
def getMV(self):
    return self.getMoveInches()
def getEHD(self):
    return self.getEquivalentHitDice()

# Basic mutators
def setAlignment(self, align):
    self.alignment = align
def clearTimesMeleed(self):
    self.timesMeleed = 0
def incTimesMeleed(self):
    self.timesMeleed +=1

## Null methods for Character inheritance
def getArmor(self):
    return None
def getShield(self):
    return None
def getWeapon(self):
    return None
def setArmor (a):
    pass
def drawBestWeapon(m):
    pass
def sheatheWeapon():
    pass
def boostMagicItemsOneLevel():
    pass
def takeAbilityDamage (a, n):
    pass
def zeroAbilityDamage():
    pass
def hasNullAbilityScore():
    return False
def hasFeat(feat):
    return False
def getSpellList():
    return None
def addXP(xp):
    pass
def getSweepRate():
    return 0       
    

In [121]:
## Arena Functions
def setBaseArmorFromInt(code):
    if code == 0:
        baseArmorType = None
    if code == 1:
        baseArmorType = armType(3) #Leather
    if code == 2:
        baseArmorType = armType(2) #Chaing
    if code == 3:
        baseArmorType = armType(1) #Plain
        

def recruitNewFighters():
    pass
        
def reportStart():
    print("Settings: ")
    print("fightManVsMonster " + str(fightManVsMonster))
    print("numFighters " + str(numFighters))
    print("fights/year " + str(fightsPerYear ))
    print("party size " + str(fighterPartySize))
    print("treasure by " + str(useMonsterTreasureType))


reportStart()

testrun = 1
#for encounter in range(NUM_ENCOUNTERS):
for year in range(1, testrun+1):    
    print(year)
    for i in range(fightsPerYear):
        print(i)
        def runOneCycle():
            recruitNewFighters()
#fighterList.shuffleMembers()
#fightDuels()
#fighterList.bringOutYourDead()
#fighterList.clearFallen()
#fighterList.healAll()
    


Settings: 
fightManVsMonster False
numFighters 100
fights/year 12
party size 1
treasure by False
1
0
1
2
3
4
5
6
7
8
9
10
11


In [144]:
banana = Armor(1,1,100,2)
print(banana.name)
print(banana.getArmorType())
print(banana.getBaseArmor())
junk = banana.isMetal()
print(junk)

armType.Plate
1
1
True


In [131]:
poo = makeType(armType(1))
poo
poo.baseArmor
Alignment.getFromChar("L")

<Alignment.Lawful: 1>

In [147]:
banana.setMagicBonus(7)

In [148]:
banana.magicBonus

5